# Milestone 03
# Peter Lorenz

## 0. Preliminaries

Import the required libraries:

In [11]:
import sys
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.utils import class_weight
from tensorflow import keras

Set global options:

In [2]:
# Display plots inline
%matplotlib inline

# Display multiple cell outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Suppress scientific notation
np.set_printoptions(suppress=True)
np.set_printoptions(precision=3)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

# Do not truncate numpy arrays
np.set_printoptions(threshold=sys.maxsize)

Declare constants:

Declare utility functions:

## 1. Split data from Milestone 1 into training and testing
In this section, we split the prepared data from Milestone 1 into training and test data sets. But first we must reload and clean this data following our procedure in Milestone 1.

### Read and clean data (from Milestone 1)
Here we follow the steps taken in Milestone 1 to prepare our data for modeling. Commentary is kept to a minimum as these matters have already been discussed in Milestone 1. Also, cell output is kept to the minimum necessary to confirm that the code is functioning as expected.

In [87]:
# Internet location of the data set and labels
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/secom/secom.data"
labels_url = "https://archive.ics.uci.edu/ml/machine-learning-databases/secom/secom_labels.data"

# Download sensor data and labels into a dataframe object, specify python engine for regex
sensor_data = pd.read_csv(url, sep='\s{1,}', engine='python')
sensor_labels_data = pd.read_csv(labels_url, sep='\s{1,}', engine='python')

# Generate index-based column names for the sensor data set
sensor_data.columns = list('s' + str(idx + 1) for idx in range(0, sensor_data.shape[1]))

# Assign column names to the labels
sensor_labels_data.columns = ['result', 'date', 'time']

# Save the original data frame for future reference as we modify its contents
sensor_data_orig = sensor_data

# Confirm that data set and labels are loaded
print('Sensor data set:')
sensor_data.shape
sensor_data.head()

print('Sensor labels:')
sensor_labels_data.shape
sensor_labels_data.head()

Sensor data set:


(1566, 590)

,s1,s2,s3,s4,s5,s6,s7,s8,s9,s10,...,s581,s582,s583,s584,s585,s586,s587,s588,s589,s590
0,3095.780,2465.140,2230.422,1463.661,0.829,100.000,102.343,0.125,1.497,-0.001,...,0.006,208.204,0.502,0.022,0.005,4.445,0.010,0.020,0.006,208.204
1,2932.610,2559.940,2186.411,1698.017,1.510,100.000,95.488,0.124,1.444,0.004,...,0.015,82.860,0.496,0.016,0.004,3.175,0.058,0.048,0.015,82.860
2,2988.720,2479.900,2199.033,909.793,1.320,100.000,104.237,0.122,1.488,-0.012,...,0.004,73.843,0.499,0.010,0.003,2.054,0.020,0.015,0.004,73.843
3,3032.240,2502.870,2233.367,1326.520,1.533,100.000,100.397,0.123,1.503,-0.003,...,nan,nan,0.480,0.477,0.104,99.303,0.020,0.015,0.004,73.843
4,2946.250,2432.840,2233.367,1326.520,1.533,100.000,100.397,0.123,1.529,0.017,...,0.005,44.008,0.495,0.019,0.004,3.828,0.034,0.015,0.005,44.008


Sensor labels:


(1566, 3)

,result,date,time
0,-1,"""19/07/2008","12:32:00"""
1,1,"""19/07/2008","13:17:00"""
2,-1,"""19/07/2008","14:43:00"""
3,-1,"""19/07/2008","15:22:00"""
4,-1,"""19/07/2008","17:53:00"""


Drop columns with more than 5% NaN:

In [88]:
# Count NaN's per column
df_na = sensor_data.isna().sum()

# Identify columns above cutoff of 5% NaN's
nan_5_pct = df_na[df_na > 0.05 * sensor_data.shape[0]]

# Drop columns with more than 5% NaN's
sensor_data = sensor_data.drop(list(nan_5_pct.index), axis=1)
sensor_data.shape

(1566, 538)

Impute fields with NaN in the remaining columns:

In [89]:
# Impute and replace missing values using column median
sensor_data = sensor_data.replace('?', 
                                  np.NaN).apply(lambda x: x.fillna(x.median()))

Remove columns with zero variance:

In [90]:
# Identify columns with zero variance
zero_variance_cols = np.array(sensor_data.columns[sensor_data.var() == 0])

# Drop columns with zero variance
sensor_data = sensor_data.drop(zero_variance_cols, axis=1)

Remove columns consisting of more than 20% zeros:

In [91]:
# Count zeroes per column
zero_counts = sensor_data[sensor_data == 0].count(axis=0)

# Identify columns above cutoff of 20% zero
zero_20_pct = zero_counts[zero_counts > 0.2 * sensor_data.shape[0]]

# Drop columns with more than 20% zeros
sensor_data = sensor_data.drop(list(zero_20_pct.index), axis=1)
sensor_data.shape

(1566, 401)

Our data set is now almost ready for modeling. We deal with class inbalance and feature standardization after splitting the data into test and training sets.

### Feature standardization
We choose RobustScaler over StandardScaler due to the skewness of a significant number of features in the data set, as determined in Milestone 1. Because StandardScaler must compute the mean and standard deviation, it is susceptible to outliers. On the other hand, RobustScaler is based on percentiles and, hence, is less susceptible to outliers. We now apply RobustScaler to our training set:

In [92]:
# Scale data
scaler = RobustScaler()
sensor_data = pd.DataFrame(scaler.fit_transform(sensor_data), 
                           columns=sensor_data.columns)

# Display scaled data set
sensor_data.head()

,s1,s2,s3,s4,s5,s7,s8,s9,s10,s11,...,s577,s578,s583,s584,s585,s586,s587,s588,s589,s590
0,0.937,-0.398,0.794,0.352,-0.973,0.126,0.852,0.332,0.042,-1.322,...,-1.049,-0.523,0.380,1.735,1.900,1.706,-0.769,0.546,0.452,1.937
1,-0.875,0.705,-0.396,0.815,0.386,-0.916,0.630,-0.170,0.281,0.078,...,0.953,-0.814,-0.983,0.388,0.300,0.422,2.674,3.464,3.290,0.156
2,-0.252,-0.226,-0.055,-0.741,0.007,0.414,-0.259,0.252,-0.578,-0.322,...,0.271,-0.938,-0.268,-0.714,-1.100,-0.711,-0.021,0.010,-0.065,0.028
3,0.231,0.041,0.873,0.082,0.432,-0.170,0.407,0.393,-0.094,-0.661,...,0.071,-0.510,-4.514,94.449,100.900,97.651,-0.021,0.010,-0.065,0.028
4,-0.724,-0.773,0.873,0.082,0.432,-0.170,0.407,0.636,0.937,0.443,...,0.088,-0.008,-1.184,1.041,0.800,1.082,0.966,0.031,0.194,-0.396


### Split data into training and test
We now split the data into training and test data sets, reserving ten percent of the rows for testing (157 rows) and using the rest to train our models. We choose this relatively high number to ensure that a sufficient number of positives exist in the test data:

In [93]:
# Split data into training and test
X_train, X_test, y_train, y_test = \
    train_test_split(sensor_data, 
                     sensor_labels_data['result'], 
                     test_size = 0.2,
                     random_state = 0)

# Describe training and test
print("Training data has {} rows.".format(X_train.shape[0]))
print("Test data has {} rows.".format(X_test.shape[0]))

Training data has 1252 rows.
Test data has 314 rows.


Now that the test data has been isolated, we can deal with class inbalance and feature standardization in the training data.

### Balance classes using class weights
To address class inbalance we are not using oversampling as in previous iterations of this project. Instead, we pass a dictionary of class weights to the fit() method on the model (below).

We now turn to our results from Milestone 1 to select the features for our models.

### Feature selection
In Milestone 1, we applied four different methods to select the best features from our large set of features. Since stepwise (forward) selection produced the features with the most correlation with the other methods, we will go with the twenty best features chosen by this method:

1. s22
1. s34
1. s65
1. s104
1. s125
1. s130
1. s144
1. s189
1. s313
1. s438

We select these features here for use below in the models:

In [94]:
# Stepwise-selected features from Milestone 1
stepwise_features = ['s22', 's34', 's65', 's104', 's125', 's130', 's144', 's189', 's313', 's438']

Now that our training data set has been resampled and standardized and we have selected our features, we are ready to proceed to modeling.

## 1. Build a simple neural networks model
TODO

## 2. Build a DNN model
TODO

In [95]:
# Prepare class weights to balance classes
class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(y_train),
                                                 y_train)

# Create class-weights dictionary
class_weights = dict(zip([-1, 1], class_weights))

In [100]:
# Create model
model = keras.Sequential([
    keras.layers.Dense(128, activation = tf.nn.relu, input_dim = 401),
    keras.layers.Dense(128, activation = tf.nn.relu),
    keras.layers.Dense(1, activation = tf.nn.softmax)
])

# Compile model
model.compile(optimizer = 'sgd', 
              loss = 'binary_crossentropy',
              metrics = ['accuracy'])

In [101]:
# Fit model
model.fit(X_train, y_train, 
          epochs = 10, class_weight=class_weights)

Epoch 1/10
1252/1252 [==============================] - 0s 368us/sample - loss: 18.3788 - acc: 0.0727
Epoch 2/10
1252/1252 [==============================] - 0s 104us/sample - loss: 17.5709 - acc: 0.0727
Epoch 3/10
1252/1252 [==============================] - 0s 113us/sample - loss: 17.2662 - acc: 0.0727
Epoch 4/10
1252/1252 [==============================] - 0s 104us/sample - loss: 17.2776 - acc: 0.0727
Epoch 5/10
1252/1252 [==============================] - 0s 106us/sample - loss: 17.4787 - acc: 0.0727
Epoch 6/10
1252/1252 [==============================] - 0s 107us/sample - loss: 17.8821 - acc: 0.0727
Epoch 7/10
1252/1252 [==============================] - 0s 110us/sample - loss: 16.9528 - acc: 0.0727
Epoch 8/10
1252/1252 [==============================] - 0s 106us/sample - loss: 17.8189 - acc: 0.0727
Epoch 9/10
1252/1252 [==============================] - 0s 111us/sample - loss: 17.4310 - acc: 0.0727
Epoch 10/10
1252/1252 [==============================] - 0s 106us/sample - loss: 1

In [102]:
# Evaluate model
test_loss, test_acc = model.evaluate(X_test, y_test)

# Display accuracy
print('Test accuracy:', test_acc)

314/314 [==============================] - 0s 666us/sample - loss: 30.5647 - acc: 0.0414
Test accuracy: 0.041401275


## 3. Build a RNN model
TODO

## 4. Summarize your findings
TODO: Summarize your findings with examples. Explain what the manufacturer should focus on to optimize
the diaper manufacturing process.